# Case Study : Plant capacity & Supply Chain Optimization

Assume the case study data is from a car manufacture optimizing its Supply Chain network across five regions (i.e. USA, Germany, Japan, Brazil, and India). You are given the demand, manufacturing capacity (thousands of cars) for each region, and the variable and fixed costs (thousands of $US dollars). Four Pandas DataFrames demand, var_cost, fix_cost, and cap have been created for you, and printed in the console, containing the regional demand, variable production costs, fixed production costs, and production capacity. The var_cost shows the costs of producing in location i shipping to location j. Which statement is true based on the data.

In [29]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

In [30]:
cap = np.array([[ 500, 1500],
       [ 500, 1500],
       [ 500, 1500],
       [ 500, 1500],
       [ 500, 1500]])

cap = pd.DataFrame(cap, columns =['Low_Cap', 'High_Cap'], index=['USA', 'Germany', 'Japan', 'Brazil', 'India'])

cap

,Low_Cap,High_Cap
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,1500


In [31]:
fc = np.array([[6500, 9500],
       [4980, 7270],
       [6230, 9100],
       [3230, 4730],
       [2110, 3080]])

fix_cost = pd.DataFrame(fc, columns =['Low_Cap', 'High_Cap'], 
                        index=['USA', 'Germany', 'Japan', 'Brazil', 'India'])

fix_cost

,Low_Cap,High_Cap
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,3080


In [32]:
var_cost = pd.DataFrame(
                        data = np.array([[ 6, 13, 20, 12, 17],
                                            [13,  6, 14, 14, 13],
                                            [20, 14,  3, 21,  9],
                                            [12, 14, 21,  8, 21],
                                            [22, 13, 10, 23,  8]]),
                        columns = ['USA', 'Germany', 'Japan', 'Brazil', 'India'],
                        index = ['USA', 'Germany', 'Japan', 'Brazil', 'India'])

#is in 1000 USD
var_cost

,USA,Germany,Japan,Brazil,India
USA,6,13,20,12,17
Germany,13,6,14,14,13
Japan,20,14,3,21,9
Brazil,12,14,21,8,21
India,22,13,10,23,8


In [33]:
demand = pd.DataFrame(data = np.array([[2719.6],
                                        [  84.1],
                                        [1676.8],
                                        [ 145.4],
                                        [ 156.4]]),
                      columns = ['Demand (x1000 cars)'],
                     index = ['USA', 'Germany', 'Japan', 'Brazil', 'India'])

demand

,Demand (x1000 cars)
USA,2719.6
Germany,84.1
Japan,1676.8
Brazil,145.4
India,156.4


## Step 1 : Define your problem statement.

> You need to Minimize cost, so it is a LpMiniMize problem.

In [34]:
# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Step 2 : Define details relevant to the supply chain problem. 

> Plant locations, Plant sizes etc.

In [35]:
# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']

# Step 3 : Create Decision Variables : Stuff you can control.

> Plant operation status $(y_{si})$

> Quantity produced and transported from i to j $(x_{ij})$

In [36]:
x = LpVariable.dicts(name='Export_Qtty_iToj', indices=[(i,j) for i in loc for j in loc],
                     lowBound=0, cat='Continuous')

y = LpVariable.dicts(name='Operational_status_of_Plant_si', indices=[(s,i) for s in size for i in loc],
                      cat='Binary')

# Step 4 : Define Objective Function.

Minimize cost. 

Cost = Fix + Variable

>> Fixed cost is the one needed to keep plant operational

>> Variable cost is the one needed to transport/export

Lets Assume (per the magnitude) Fixed cost is overall and variable cost given is per 1000s of manufactured cars (the unit of X)

In [37]:
fix_cost

,Low_Cap,High_Cap
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,3080


In [38]:
var_cost

,USA,Germany,Japan,Brazil,India
USA,6,13,20,12,17
Germany,13,6,14,14,13
Japan,20,14,3,21,9
Brazil,12,14,21,8,21
India,22,13,10,23,8


In [39]:
model += (lpSum([x[(i,j)]*var_cost.loc[i,j] for i in loc for j in loc]) +
          lpSum([y[(s,i)]*fix_cost.loc[i,s] for s in size for i in loc]))

# Step 4 : Define Constraints. 

> Supply == Demand

> production <= Capacity

In [40]:
demand

,Demand (x1000 cars)
USA,2719.6
Germany,84.1
Japan,1676.8
Brazil,145.4
India,156.4


Supply/Production is defined by our variable x

#### For example, supply to USA is a contribution of Germany + Japan + Brazil + India production/export

##### Constraint 1 : Supply Demand balance

In [41]:
for j in loc:

  model += lpSum([x[(i,j)] for i in loc]) == demand.loc[j, 'Demand (x1000 cars)']

#### Constraint 2 : Production Capacity constraint.

> given : manufacturing capacity (thousands of cars) 

In [42]:
cap

,Low_Cap,High_Cap
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,1500


In [43]:
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(s,i)]
                                                      for s in size])

# Step 5 : SOLVE!

In [44]:
model.solve()

1

## Storing results

1. Exports/Supply

In [48]:
supply = var_cost.copy() ; supply.values[:,:] = 0

supply

,USA,Germany,Japan,Brazil,India
USA,0,0,0,0,0
Germany,0,0,0,0,0
Japan,0,0,0,0,0
Brazil,0,0,0,0,0
India,0,0,0,0,0


In [49]:
for i in loc:
  for j in loc:
    print(f'X({i},{j}) = {x[(i,j)].varValue}')

    supply.loc[i,j] = x[(i,j)].varValue



X(USA,USA) = 1500.0
X(USA,Germany) = 0.0
X(USA,Japan) = 0.0
X(USA,Brazil) = 0.0
X(USA,India) = 0.0
X(Germany,USA) = 0.0
X(Germany,Germany) = 0.0
X(Germany,Japan) = 0.0
X(Germany,Brazil) = 0.0
X(Germany,India) = 0.0
X(Japan,USA) = 0.0
X(Japan,Germany) = 0.0
X(Japan,Japan) = 1500.0
X(Japan,Brazil) = 0.0
X(Japan,India) = 0.0
X(Brazil,USA) = 1219.6
X(Brazil,Germany) = 0.0
X(Brazil,Japan) = 0.0
X(Brazil,Brazil) = 145.4
X(Brazil,India) = 0.0
X(India,USA) = 0.0
X(India,Germany) = 84.1
X(India,Japan) = 176.8
X(India,Brazil) = 0.0
X(India,India) = 156.4


In [50]:
supply

,USA,Germany,Japan,Brazil,India
USA,1500.0,0.0,0.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0
Japan,0.0,0.0,1500.0,0.0,0.0
Brazil,1219.6,0.0,0.0,145.4,0.0
India,0.0,84.1,176.8,0.0,156.4


2. Plants operational status

In [53]:
opstatus = cap.copy() ; opstatus.values[:,:] = 0

opstatus

,Low_Cap,High_Cap
USA,0,0
Germany,0,0
Japan,0,0
Brazil,0,0
India,0,0


In [54]:
for s in size:
  for i in loc:

    print(f'y[{s},{i}] = {y[(s,i)].varValue}')

    opstatus.loc[i,s] = y[(s,i)].varValue

y[Low_Cap,USA] = 0.0
y[Low_Cap,Germany] = 0.0
y[Low_Cap,Japan] = 0.0
y[Low_Cap,Brazil] = 0.0
y[Low_Cap,India] = 1.0
y[High_Cap,USA] = 1.0
y[High_Cap,Germany] = 0.0
y[High_Cap,Japan] = 1.0
y[High_Cap,Brazil] = 1.0
y[High_Cap,India] = 0.0


In [55]:
opstatus

,Low_Cap,High_Cap
USA,0,1
Germany,0,0
Japan,0,1
Brazil,0,1
India,1,0
